In [1]:
import pandas as pd
import numpy as np

In [2]:
PATH_TO_DATA = '/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/'
data_time = '20180920'

## Get red links sample

### Files needed for final processing:
### 1. 'ukwiki-20180920-pages-links.csv'
### 2. "ukwiki-" + data_time + "-id_name.csv"
### 3. '20180920-langlinks_uk_en.csv'
### 4. 'ukwiki-20180920-all_reslinks.csv'

In [3]:
df_uk_full = pd.read_csv(PATH_TO_DATA+'ukwiki-20180920-pages-links.csv')
print(df_uk_full.shape)
df_uk_full.head()

(22650865, 6)


,id,title,text_len,link_pos,link_val,link_txt
0,3,Головна сторінка,3197,0,NaN,NaN
1,13,Географія,16843,1228,Об'єкт (філософія),Об'єкт
2,13,Географія,16843,1533,Ератосфен,NaN
3,13,Географія,16843,1599,географ,NaN
4,13,Географія,16843,1611,Клавдій Птолемей,NaN


In [4]:
df_uk_red = pd.read_csv(PATH_TO_DATA+'ukwiki-20180920-all_reslinks.csv', encoding='UTF-8', quotechar="\"")

In [5]:
df_uk_red.head()

,id,link_val
0,13,Географія розвитку
1,13,Географія часу
2,13,Туристична географія
3,584,рентгенівська спектроскопія поглинання
4,584,рентгенівська фото електронна спектроскопія


In [6]:
# article names
df_uk_name = pd.read_csv(PATH_TO_DATA+"ukwiki-" + data_time + "-id_name.csv", encoding='utf-8', sep="^")

print(df_uk_name.shape)
print(df_uk_name.columns)
df_uk_name.head()

(819212, 3)
Index(['id', 'title', 'length'], dtype='object')


,id,title,length
0,3,Головна сторінка,3197
1,13,Географія,16843
2,584,Атом,35112
3,585,Мільярд,1759
4,586,Ядро,8064


In [7]:
# find maximum id among blue links
max_id = np.max(df_uk_name['id'])

# find unique titles among red links, create df with

red_links = df_uk_red['link_val'].unique()

red_links_ids = pd.DataFrame({'title': red_links, 
                               'id': np.arange(max_id+1, len(red_links)+max_id+1)})

red_links_ids.head()

,title,id
0,Географія розвитку,2817993
1,Географія часу,2817994
2,Туристична географія,2817995
3,рентгенівська спектроскопія поглинання,2817996
4,рентгенівська фото електронна спектроскопія,2817997


In [8]:
df_uk_red_ext = df_uk_red.merge(right = red_links_ids, left_on = 'link_val', right_on = 'title', how = 'right')
df_uk_red_ext = df_uk_red_ext[['id_x', 'link_val', 'id_y']]
df_uk_red_ext.columns = ['id_parent', 'red_link_name', 'red_link_id']
print(df_uk_red_ext.shape)
df_uk_red_ext.head()

(2872272, 3)


,id_parent,red_link_name,red_link_id
0,13,Географія розвитку,2817993
1,13,Географія часу,2817994
2,1928599,Географія часу,2817994
3,13,Туристична географія,2817995
4,584,рентгенівська спектроскопія поглинання,2817996


In [9]:
df_uk_red_ext.to_csv(PATH_TO_DATA+'all_uk_redlinks_with_parent_ids.csv', header=True, index=False)

In [10]:
# Load uk-en correpondences
fn = PATH_TO_DATA+'20180920-langlinks_uk_en.csv'
df_link = pd.read_csv(fn,  encoding='UTF-8', quotechar='\'')
print(df_link.shape)
print(df_link.columns)
df_link.head()

(599612, 3)
Index(['ll_from', 'll_lang', 'll_title'], dtype='object')


,ll_from,ll_lang,ll_title
0,2112530,en,! (The Dismemberment Plan album)
1,1366003,en,! (disambiguation)
2,2141483,en,!!
3,425480,en,!!!
4,848226,en,!Action Pact!


In [11]:
df_uk_red_ext = pd.read_csv(PATH_TO_DATA+'all_uk_redlinks_with_parent_ids.csv')
print(df_uk_red_ext.shape)
df_uk_red_ext.head()

(2872272, 3)


,id_parent,red_link_name,red_link_id
0,13,Географія розвитку,2817993
1,13,Географія часу,2817994
2,1928599,Географія часу,2817994
3,13,Туристична географія,2817995
4,584,рентгенівська спектроскопія поглинання,2817996


In [12]:
# add en correspondences to uk parent pages for red links
df_uk_red_corresp = df_uk_red_ext.merge(right = df_link, left_on = 'id_parent', right_on = 'll_from', how = 'left')
df_uk_red_corresp.head()

,id_parent,red_link_name,red_link_id,ll_from,ll_lang,ll_title
0,13,Географія розвитку,2817993,13.0,en,Geography
1,13,Географія часу,2817994,13.0,en,Geography
2,1928599,Географія часу,2817994,1928599.0,en,Geography (disambiguation)
3,13,Туристична географія,2817995,13.0,en,Geography
4,584,рентгенівська спектроскопія поглинання,2817996,584.0,en,Atom


In [13]:
df_uk_red_corresp = df_uk_red_corresp[pd.notnull(df_uk_red_corresp['ll_from'])]
df_uk_red_corresp.head()

,id_parent,red_link_name,red_link_id,ll_from,ll_lang,ll_title
0,13,Географія розвитку,2817993,13.0,en,Geography
1,13,Географія часу,2817994,13.0,en,Geography
2,1928599,Географія часу,2817994,1928599.0,en,Geography (disambiguation)
3,13,Туристична географія,2817995,13.0,en,Geography
4,584,рентгенівська спектроскопія поглинання,2817996,584.0,en,Atom


In [14]:
df_uk_red_corresp = df_uk_red_corresp[['id_parent','red_link_name', 'red_link_id', 'll_title']]

In [15]:
df_uk_red_corresp.to_csv(PATH_TO_DATA+'df_allukredlinks_withentitle.csv', header=True, index=False)

In [16]:
df_uk_red_corresp = df_uk_red_corresp[['id_parent','red_link_name', 'red_link_id']]
df_uk_red_corresp = df_uk_red_corresp.sort_values(by = 'red_link_id')
df_uk_red_corresp = df_uk_red_corresp.drop_duplicates(keep = 'first')
df_uk_red_corresp = df_uk_red_corresp.dropna()
df_uk_red_corresp

,id_parent,red_link_name,red_link_id
0,13,Географія розвитку,2817993
1,13,Географія часу,2817994
2,1928599,Географія часу,2817994
3,13,Туристична географія,2817995
4,584,рентгенівська спектроскопія поглинання,2817996
5,584,рентгенівська фото електронна спектроскопія,2817997
11,2328123,мезолімбічний шлях,2817999
10,1209913,мезолімбічний шлях,2817999
8,42037,мезолімбічний шлях,2817999
9,427773,мезолімбічний шлях,2817999


In [17]:
# for every red link calculate number of incoming links
df_red_links_most_common = df_uk_red_corresp['red_link_id'].value_counts().reset_index()
df_red_links_most_common.head()

,index,red_link_id
0,2863300,941
1,2969411,848
2,4169378,760
3,3060807,687
4,3512597,639


In [18]:
df_red_links_most_common.columns = ['red_link_id_', 'count']
df_red_links_most_common.head()

,red_link_id_,count
0,2863300,941
1,2969411,848
2,4169378,760
3,3060807,687
4,3512597,639


In [19]:
df_red_links_most_common = df_red_links_most_common[(df_red_links_most_common['count'] >= 25)]
df_red_links_most_common

,red_link_id_,count
0,2863300,941
1,2969411,848
2,4169378,760
3,3060807,687
4,3512597,639
5,2826629,502
6,2826628,502
7,2826630,502
8,2826631,495
9,2825668,474


In [20]:
result = df_uk_red_corresp.merge(right = df_red_links_most_common,
                                 left_on = 'red_link_id', right_on = 'red_link_id_', how = 'inner')

In [21]:
result

,id_parent,red_link_name,red_link_id,red_link_id_,count
0,2579654,Каунт Бейсі,2818114,2818114,43
1,2543496,Каунт Бейсі,2818114,2818114,43
2,2543162,Каунт Бейсі,2818114,2818114,43
3,2538289,Каунт Бейсі,2818114,2818114,43
4,2581857,Каунт Бейсі,2818114,2818114,43
5,2530258,Каунт Бейсі,2818114,2818114,43
6,2525341,Каунт Бейсі,2818114,2818114,43
7,2389752,Каунт Бейсі,2818114,2818114,43
8,2535297,Каунт Бейсі,2818114,2818114,43
9,2590135,Каунт Бейсі,2818114,2818114,43


In [22]:
result_red_links_uk = result['red_link_name'].unique()
df_result_red_links_uk = pd.DataFrame(result_red_links_uk, columns=['red_link_name'])

# save red links to file
df_result_red_links_uk.to_csv(PATH_TO_DATA+'df_sample_red_links_uk.csv', header=True, index=False)

In [23]:
# results in files:
# 1. 'all_uk_redlinks_with_parent_ids.csv',
# 2. 'df_allukredlinks_withentitle.csv',
# 3. 'df_sample_red_links_uk.csv'